## Introdução

O projeto em questão está focado na análise de dados relacionados ao mercado de livros, particularmente durante a pandemia de coronavírus, que transformou os hábitos de consumo de conteúdo, levando muitas pessoas a se dedicarem à leitura. Para entender esse fenômeno, um banco de dados de uma startup concorrente no mercado de livros foi fornecido, contendo informações sobre livros, autores, editoras, avaliações dos leitores e resenhas dos usuários.

A análise desses dados nos permitirá gerar insights valiosos para ajudar no desenvolvimento de um novo produto para os amantes de livros. As consultas SQL a serem realizadas são divididas em várias tarefas, que nos ajudarão a responder perguntas específicas sobre o comportamento dos leitores, as editoras, e a recepção dos livros no mercado.

### Tarefa 1: Encontrar o número de livros lançados depois de 1 de janeiro de 2000

Nesta tarefa, o objetivo é entender o volume de lançamentos de livros após o ano de 2000. Sabemos que a indústria editorial passou por transformações significativas após esse período, com novas editoras, tecnologias e mudanças nos padrões de consumo. A partir da análise dessa data de lançamento, podemos obter uma visão sobre o comportamento do mercado editorial ao longo do tempo.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = { 
                'user': 'practicum_student', # username
                'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
                'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
                'port': 5432, # connection port
                'db': 'data-analyst-final-project-db' # the name of the database
            }

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config ['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# Consulta SQL para contar o número de livros lançados após 1 de janeiro de 2000
query = """
    SELECT COUNT(*) 
    FROM books
    WHERE publication_date > '2000-01-01';
"""

# Executando a consulta e carregando o resultado no DataFrame
result = pd.read_sql(query, con = engine)

# Exibindo o número de livros lançados após 1º de janeiro de 2000
num_livros_lancados = result.iloc[0, 0]  # O resultado vem em uma célula no DataFrame
print(f'Número de livros lançados após 1º de janeiro de 2000: {num_livros_lancados}')


Número de livros lançados após 1º de janeiro de 2000: 819


<div class="alert alert-block alert-success">
<b> Comentário: </b> <a class="tocSkip"></a>

    
- A quantidade de livros lançadas após Jan-2000 foi calculada
</div>

### Tarefa 2: Encontrar o número de avaliações e a classificação média para cada livro

Aqui, o foco é analisar a recepção dos livros pelos leitores. Com isso, podemos identificar os livros que receberam mais avaliações, além de calcular a média das classificações para cada livro. Isso nos ajudará a entender a popularidade e a qualidade dos livros, além de fornecer uma visão mais precisa sobre o que os leitores pensam sobre os títulos disponíveis.

In [3]:
# Consulta SQL para encontrar o número de avaliações e a média das classificações por livro
query = """
    SELECT 
        book_id, 
        COUNT(rating_id) AS num_avaliacoes,
        AVG(rating) AS media_classificacao
    FROM ratings
    GROUP BY book_id;
"""

# Executando a consulta e carregando o resultado no DataFrame
avaliacoes = pd.read_sql(query, engine)

# Exibindo as primeiras linhas do DataFrame resultante
print(avaliacoes.head())



   book_id  num_avaliacoes  media_classificacao
0      652               2             4.500000
1      273               2             4.500000
2       51              12             4.250000
3      951               2             4.000000
4      839               7             4.285714


### Tarefa 3: Identificar a editora que lançou o maior número de livros com mais de 50 páginas

Nesta análise, queremos focar nas editoras que publicaram livros com mais de 50 páginas, excluindo assim brochuras e livros de menor porte. Saber quais editoras dominam o mercado de livros mais longos pode ser útil para entender quais editoras estão investindo em títulos mais robustos. Isso também pode ajudar a destacar editoras que se dedicam mais ao público ávido por leitura.

In [4]:
import pandas as pd

# Consulta SQL para identificar a editora com o maior número de livros com mais de 50 páginas
query = """
    SELECT 
        p.publisher, 
        COUNT(b.book_id) AS num_livros
    FROM books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY num_livros DESC
    LIMIT 1;
"""

# Executando a consulta e carregando o resultado no DataFrame
resultado = pd.read_sql(query, engine)

# Exibindo o resultado
print("Editora com o maior número de livros com mais de 50 páginas:")
print(resultado)

Editora com o maior número de livros com mais de 50 páginas:
       publisher  num_livros
0  Penguin Books          42


### Tarefa 4: Identificar o autor com a média mais alta de classificação de livros (considerando apenas livros com pelo menos 50 classificações)

Neste caso, o objetivo é determinar o autor mais bem classificado, levando em consideração apenas os livros que receberam uma quantidade significativa de avaliações, ou seja, pelo menos 50 classificações. Isso nos ajudará a identificar os autores que são consistentemente bem avaliados pelos leitores.

In [5]:
# Consulta SQL para identificar o autor com a maior média de classificação, considerando livros com pelo menos 50 classificações
query = """
    SELECT 
        a.author, 
        AVG(r.rating) AS media_classificacao
    FROM ratings r
    JOIN books b ON r.book_id = b.book_id
    JOIN authors a ON b.author_id = a.author_id
    GROUP BY a.author, b.book_id
    HAVING COUNT(r.rating_id) >= 50
    ORDER BY media_classificacao DESC
    LIMIT 1;
"""

# Executando a consulta e carregando o resultado no DataFrame
resultado = pd.read_sql(query, con = engine)

# Exibindo o resultado
print("Autor com a maior média de classificação (considerando livros com pelo menos 50 classificações):")
print(resultado)


Autor com a maior média de classificação (considerando livros com pelo menos 50 classificações):
                       author  media_classificacao
0  J.K. Rowling/Mary GrandPré             4.414634


<div class="alert alert-block alert-success">
<b> Comentário: </b> <a class="tocSkip"></a>

    
Parabéns, todas as tarefas até aqui foram realizadas corretamente :D

</div>

### Tarefa 5: Encontrar o número médio de avaliações entre usuários que classificaram mais do que 50 livros

Esta análise se concentra no comportamento de usuários que são mais ativos em termos de avaliações. Queremos calcular a média de avaliações feitas por aqueles que avaliaram mais de 50 livros, oferecendo insights sobre o perfil dos leitores mais engajados. Esses usuários podem ser considerados como formadores de opinião ou simplesmente leitores mais dedicados.

In [7]:
# Consulta SQL para calcular a média da contagem de avaliações de texto (reviews) feitas por usuários que avaliaram mais de 50 livros
query = """
    SELECT 
        AVG(review_count) AS avg_reviews
    FROM (
        SELECT 
            COUNT(*) AS review_count
        FROM reviews
        WHERE username IN (
            SELECT
                username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50
        )
        GROUP BY username
    ) AS review_counts;
"""

# Executando a consulta e carregando o resultado no DataFrame
resultado = pd.read_sql(query, con=engine)

# Exibindo o resultado
print("Número médio de avaliações de texto feitas por usuários que avaliaram mais de 50 livros:")
print(resultado)


Número médio de avaliações de texto feitas por usuários que avaliaram mais de 50 livros:
   avg_reviews
0    24.333333


## Conclusão:

Esse projeto ajudou a entender melhor o comportamento dos leitores e as tendências do mercado editorial. Analisando os dados, conseguimos identificar alguns padrões interessantes:

- Lançamentos de livros: Depois de 2000, houve um bom número de lançamentos de livros (819 livros), mostrando que a indústria continuou crescendo.

- Avaliações dos livros: Livros que receberam mais avaliações e têm boas notas tendem a ser mais populares entre os leitores. Isso pode ser útil para saber quais livros têm mais chances de sucesso.

- Editoras de livros longos: Editoras como a Penguin Books, que publicam livros com mais de 50 páginas, estão dominando o mercado de livros mais robustos. Isso pode ser um sinal de que há um público fiel para esse tipo de livro.

- Autores bem avaliados: Autores como J.K. Rowling/Mary GrandPré têm uma média de classificação alta, o que mostra que são muito bem recebidos pelo público.

- Leitores mais ativos: Os usuários que avaliam muitos livros (mais de 50) fazem em média 54 avaliações cada. Esses leitores são valiosos, porque ajudam a influenciar outros e engajam mais com os livros.


## Recomendações:

Com base nesses insights, editoras podem focar em promover livros com boas classificações e mais avaliações, investir em títulos mais longos e apostar em autores consagrados. Além disso, engajar leitores mais ativos pode ser uma boa estratégia para aumentar o sucesso no mercado.

Esses dados são úteis para entender como os leitores estão interagindo com os livros e como as editoras podem melhorar suas ofertas para se destacar ainda mais no mercado.

<div class="alert alert-block alert-success">

<b>Comentário: </b> <a class="tocSkip"></a>



Conclusão detalhada e baseada em dados.

</div>

<div class="alert alert-block alert-success">

<b>Comentário geral v2: </b> <a class="tocSkip"></a>
    
Bruuu o trabalho ficou impecável! Muuuito bom! Fico triste que será a nossa última revisão, mas feliz pelo seu avanço e desenvovimento! Pode contar comigo pra qualquer ajuda (Dentro ou fora da Tripleten :D)

Te desejo sucesso na sua jornada!🍀